In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [96]:

class multi_head(nn.Module):
    def __init__(self):
        super(multi_head, self).__init__()
        self.A = nn.ModuleList([nn.Linear(32, 1) for _ in range(2)])
        self.weight_init()

    def weight_init(self):
        for i in range(2):
            nn.init.xavier_normal_(self.A[i].weight)
            self.A[i].bias.data.fill_(0.0)
    
    def attn_summary(self, features):
        features_attn = []
        k = features[0].shape[0]
        for i in range(2):
            features_attn.append((self.A[i](features[i].squeeze())))
        features_attn = F.softmax(torch.cat(features_attn), dim=-1).unsqueeze(1)
        features = torch.cat(features)
        features = (features * features)
        
        # k = features[i].shape[0]
        print(k)
        features = torch.cat((features[:k],features[k:]),axis=1)
        
        print(features.shape)
        print(features)
        # features = features.reshape(-1,64)
        return features, features_attn

# transposed_tensor = input_tensor.transpose(0, 1)

# # Repeat each row of the transposed tensor
# output_tensor = transposed_tensor.unsqueeze(1).expand(-1, len(input_tensor), -1).reshape(-1, len(input_tensor[0]))

# print(output_tensor)


    def forward(self,x):
        # x1,x2,x3,x4 = x[:,:16], x[:,16:32], x[:,32:48], x[:,48:]
        x1,x2 = x[:,:32], x[:,32:]

        # results = self.attn_summary([x1,x2,x3,x4])
        results = self.attn_summary([x1,x2])

        return results
        
    

In [97]:
a = multi_head()

b = torch.FloatTensor([[i for i in range(64)] for j in range(5)])

aa, bb = a(b)
# print(b.shape)
# print(aa.shape)
# aa

5
torch.Size([5, 64])
tensor([[0.0000e+00, 1.0000e+00, 4.0000e+00, 9.0000e+00, 1.6000e+01, 2.5000e+01,
         3.6000e+01, 4.9000e+01, 6.4000e+01, 8.1000e+01, 1.0000e+02, 1.2100e+02,
         1.4400e+02, 1.6900e+02, 1.9600e+02, 2.2500e+02, 2.5600e+02, 2.8900e+02,
         3.2400e+02, 3.6100e+02, 4.0000e+02, 4.4100e+02, 4.8400e+02, 5.2900e+02,
         5.7600e+02, 6.2500e+02, 6.7600e+02, 7.2900e+02, 7.8400e+02, 8.4100e+02,
         9.0000e+02, 9.6100e+02, 1.0240e+03, 1.0890e+03, 1.1560e+03, 1.2250e+03,
         1.2960e+03, 1.3690e+03, 1.4440e+03, 1.5210e+03, 1.6000e+03, 1.6810e+03,
         1.7640e+03, 1.8490e+03, 1.9360e+03, 2.0250e+03, 2.1160e+03, 2.2090e+03,
         2.3040e+03, 2.4010e+03, 2.5000e+03, 2.6010e+03, 2.7040e+03, 2.8090e+03,
         2.9160e+03, 3.0250e+03, 3.1360e+03, 3.2490e+03, 3.3640e+03, 3.4810e+03,
         3.6000e+03, 3.7210e+03, 3.8440e+03, 3.9690e+03],
        [0.0000e+00, 1.0000e+00, 4.0000e+00, 9.0000e+00, 1.6000e+01, 2.5000e+01,
         3.6000e+01, 4.9000e+

In [80]:
import torch

# Input tensor
input_tensor = torch.tensor([[1, 1, 1, 1, 1],
                             [2,2,2,2,2],
                             [3,3,3,3,3],
                             [1, 1, 1, 1, 1],
                             [2,2,2,2,2],
                             [3,3,3,3,3]])

torch.cat((input_tensor[:3], input_tensor[3:]),axis=1)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])